# UNIFR API EPUCK : Object Detection Methods

## Training the model with custom images 

This notebook will guide you through the different steps if you want to train the YOLO model on your own dataset, to then use the weights with the e-puck2 and the UNIFR_API_EPUCK.

Note that the model implemented in the API was scrapped to the maximum, to be as light as possible and run on low-end laptops. So all the training methods and associated possibilities have been removed. Therefore, we will have to use the official released version and copy the weight file. 

The blog: https://blog.paperspace.com/train-yolov5-custom-data/ is a good documentation if necessary.

In [1]:
import glob, os
import random

# Step 1: Set up the environment

Start by cloning the YOLOv5 repository:

<p style="background:black">
<code style="background:black;color:white">git clone https://github.com/ultralytics/yolov5
</code>
</p>

Then I'd suggest creating a new environment with conda or virtualenv, depending on what you use.

Once done, install the requirements

<p style="background:black">
<code style="background:black;color:white">pip install -r yolov5/requirements.txt
</code>
</p>

While you're at it, directly install the API as well


<p style="background:black">
<code style="background:black;color:white">pip install unifr_api_epuck
</code>
</p>

Finally, move this notebook inside the /yolov5 directory


# Step 2: Create the dataset

Create a folder to store the pictures you'll take with the robot


In [2]:
!mkdir custom_dataset

Then a proposition for a pictures capturing controller. An image will be taken every so often. The LED turning on indicates that the robot is about to take a picture. 

Note:

- The bigger the dataset, the better

- Try to cover all the possible situation (background, lighting, number of object in the field of view, etc...)

- Try to balance the classes, about the same number of each class should appear

In [ ]:
from unifr_api_epuck import wrapper

#Adapt the IP to your robot
MY_IP = "192.168.1.12"
robot = wrapper.get_robot(MY_IP)

robot.init_camera("custom_dataset")

# Get the last image number to not overwrite
n = 0
while os.path.exists("custom_dataset/image_{:05}.bmp".format(n)):
    n += 1


picture = 0
counter = 0

#Takes only 50 images, useful to change robot, situation. lighting, etc. 
while robot.go_on() and picture < 10:

    if counter%100 == 0:

        robot.enable_led(4)

    elif counter%100 == 10:

        robot.take_picture("image_{:05}".format(n))
        robot.disable_led(4)
        picture += 1
        n += 1

    counter += 1

robot.clean_up()

# Step 3: Label the images

Now you have to label all the images, for each images, a corresponding .txt file must be created. 

- For Mac and Windows: HyperLabel is apparently recommended

- For Linux, I used labelimg

The choice is ultimetaly up to you

# Step 4: Organize and prepare for training

Once your "custom_dataset/" folders looks like:

- image_00001.bmp
- image_00001.txt
- image_00002.bmp
- image_00002.txt
- ...

You can run the following cell to organize your data

In [ ]:
# put your own path here
dataset_path = 'custom_dataset'

# Percentage of images to be used for the validation set
percentage_test = 20
!mkdir training
!mkdir training/data
!mkdir training/data/images
!mkdir training/data/labels
!mkdir training/data/images/train
!mkdir training/data/images/valid
!mkdir training/data/labels/train
!mkdir training/data/labels/valid
# Populate the folders
p = percentage_test/100
for pathAndFilename in glob.iglob(os.path.join(dataset_path, "*.bmp")):  
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))
    if random.random() <=p :
        os.system(f"cp {dataset_path}/{title}.bmp training/data/images/valid")
        os.system(f"cp {dataset_path}/{title}.txt training/data/labels/valid")
    else:
        os.system(f"cp {dataset_path}/{title}.bmp training/data/images/train")
        os.system(f"cp {dataset_path}/{title}.txt training/data/labels/train")

Then in the training directory, you need to move the correct model.yaml

In [ ]:
os.system(f"cp models/yolov5m.yaml training")

## WARNING

At the top of this file, change the variable nc to your number of class.

Then create a second file, "dataset.yaml", with information about the paths, the number and the name of classes:

```

# train and val datasets (image directory or *.txt file with image paths)
train: training/data/images/train/
val: training/data/images/valid/
# number of classes
nc: 6
# class names
names: ['Red Block','Black Block','Black Ball','Blue Block','E-puck','Green Block']

```

## WARNING

This second "dataset.yaml" file must also be in the directory "/training"

Once your training folder contains:
 - data
 - dataset.yaml
 - yolov5m.yaml

With both .yaml files modified in regard to your classes, you can continue.

# Step 5: Train the model

Simply run the following cell:

Adapt the batch size to your GPU (with a RTX 3080, I was able to use 128, it depending on your setup)

In [ ]:
!python train.py --img 160 --batch 64 --epochs 300 --data training/dataset.yaml --cfg training/yolov5m.yaml --weights yolov5m.pt

# Step 6: Recover the weight file



Once the training is finish, you can go in "/runs/training/exp_current"

Here you can see different plot and informations about the training.

In the "/weights" folder, you'll find the file: "best.pt"

This is the file you have to copy, and can later on use when calling the function:

robot.initiate_model("best.pt")